In [1]:
# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tfw
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
#from google.colab import drive
#from google.colab import files

In [ ]:
class CustomCallback(tfw.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs:
            logs['accuracy'] = float(logs.get('accuracy', 0))
            logs['loss'] = float(logs.get('loss', 0))
        super().on_epoch_end(epoch, logs)

# Criar o callback para salvar o modelo apenas quando houver melhoria
checkpoint_callback = ModelCheckpoint(
    os.path.join(output_dir, "best_model.h5"),
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

In [8]:
#drive.mount('/content/drive')
# Defina o caminho para a pasta
path = "C:\\Users\\PC\\Documents\\documents\\ifam\\TETI\\Listas_Atividades\\treinamento_efficieentNet\\produtos_regionais"
# Configurações do TensorBoard
log_dir = "C:/Users/PC/Documents/documents/ifam/TETI/Listas_Atividades/treinamento_efficieentNet/logs"

# Criar o callback do TensorBoard com ajustes de frequência
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=False,
    update_freq='epoch',  # Pode ser 'batch' para mais detalhes
)

In [9]:
# Verifique o conteúdo da pasta
os.listdir(path)

['test', 'train']

In [4]:
# Configurações de Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Para dividir em treino e validação
)

In [5]:
 # Gerando os dados de treino e validação
train_generator = datagen.flow_from_directory(
    directory=path + 'train',
    target_size=(224, 224),  # Redimensiona as imagens para 224x224 pixels
    class_mode='categorical',  # Codificação one-hot para rótulos
    subset='training'  # Subconjunto de treinamento
)

Found 136 images belonging to 2 classes.


In [6]:
validation_generator = datagen.flow_from_directory(
    directory=data_dir + 'train',
    target_size=(224, 224),  # Mesmo redimensionamento
    class_mode='categorical',  # Mesmo tipo de rótulo
    subset='validation'  # Subconjunto de validação
)

Found 34 images belonging to 2 classes.


In [7]:
# Definindo hiperparâmetros
epochs_options = [10, 20]
batch_size_options = [16, 32]
learning_rate_options = [0.001, 0.0001]
optimizers_options = [Adam, SGD]

In [ ]:
# Armazenando resultados
results = []

# Loop para treinar modelos com diferentes combinações de hiperparâmetros
for epochs in epochs_options:
    for batch_size in batch_size_options:
        for learning_rate in learning_rate_options:
            for optimizer in optimizers_options:
                try:
                    # Configurando os geradores de dados
                    train_generator.batch_size = batch_size
                    validation_generator.batch_size = batch_size
    
                    # Criando o modelo
                    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
                    x = base_model.output
                    x = GlobalAveragePooling2D()(x)
                    predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)
                    model = Model(inputs=base_model.input, outputs=predictions)
    
                    # Compilando o modelo
                    model.compile(optimizer=optimizer(learning_rate=learning_rate), 
                                  loss='categorical_crossentropy', 
                                  metrics=['accuracy'])
    
                    # Treinando o modelo
                    history = model.fit(
                        train_generator,
                        epochs=epochs,
                        validation_data=validation_generator,
                        callbacks=[tensorboard_callback]
                    )
    
                    # Avaliando o modelo
                    final_loss, final_accuracy = model.evaluate(validation_generator)
    
                    # Salvando apenas resultados e hiperparâmetros
                    results.append({
                        'epochs': epochs,
                        'batch_size': batch_size,
                        'learning_rate': learning_rate,
                        'optimizer': optimizer.__name__,
                        'loss': final_loss,
                        'accuracy': final_accuracy,
                    })
    
                    # Garantir que o diretório exista
                    output_dir = "C:/Users/PC/Documents/documents/ifam/TETI/aula_11_11/datasets/saved_models"
                    os.makedirs(output_dir, exist_ok=True)
                    
                    # Salvar o modelo no diretório especificado
                    model_name = f"model_epochs{epochs}_batch{batch_size}_lr{learning_rate}_{optimizer.__name__}.h5"
                    model.save(os.path.join(output_dir, model_name), save_format='h5')
                except Exception as e:
                    print(f"Erro ao treinar modelo com hiperparâmetros {epochs}, {batch_size}, {learning_rate}, {optimizer}: {e}")


Epoch 1/10
9/9 [==============================] - 33s 3s/step - loss: 0.4595 - accuracy: 0.7794 - val_loss: 0.6968 - val_accuracy: 0.5000
Epoch 2/10
9/9 [==============================] - 35s 4s/step - loss: 0.1782 - accuracy: 0.9265 - val_loss: 0.7074 - val_accuracy: 0.5000
Epoch 3/10
9/9 [==============================] - 32s 3s/step - loss: 0.1679 - accuracy: 0.8971 - val_loss: 0.7171 - val_accuracy: 0.5000
Epoch 4/10
9/9 [==============================] - 38s 4s/step - loss: 0.1794 - accuracy: 0.9485 - val_loss: 0.7220 - val_accuracy: 0.5000
Epoch 5/10
3/9 [=========>....................] - ETA: 25s - loss: 0.0647 - accuracy: 0.9792

In [ ]:
# Convertendo resultados em DataFrame
results_df = pd.DataFrame(results)
# Após o loop
results_df.to_csv('results.csv', index=False)

In [ ]:
# Encontrando o modelo com maior acurácia
best_model_row = results_df.loc[results_df['accuracy'].idxmax()]
print("Melhor Modelo:")
print(best_model_row)
best_model_filename = f"model_epochs{best_model_row['epochs']}_batch{best_model_row['batch_size']}_lr{best_model_row['learning_rate']}_{best_model_row['optimizer']}.h5"
print(f"Carregando o modelo salvo: {best_model_filename}")
# Carregando o melhor modelo
best_model = load_model(best_model_filename)

In [ ]:
# Exibindo a tabela de resultados
print(results_df)

In [ ]:
# Gerando gráficos de acurácia e perda
for result in results:
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(result['history'].history['accuracy'], label='Acurácia Treino')
    plt.plot(result['history'].history['val_accuracy'], label='Acurácia Validação')
    plt.title('Acurácia do Modelo')
    plt.xlabel('Ép ocas')
    plt.ylabel('Acurácia')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(result['history'].history['loss'], label='Perda Treino')
    plt.plot(result['history'].history['val_loss'], label='Perda Validação')
    plt.title('Perda do Modelo')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()
    plt.show()

In [ ]:
# Gerando a matriz de confusão para o melhor modelo
y_true = validation_generator.classes
y_pred = best_model['model'].predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Criando a matriz de confusão
cm = confusion_matrix(y_true, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=validation_generator.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Gerador para o conjunto de teste
test_datagen = ImageDataGenerator(rescale=1./255)  # Apenas normalização, sem augmentação

test_generator = test_datagen.flow_from_directory(
    directory=data_dir + 'test',  # Substitua pelo caminho do diretório de teste
    target_size=(224, 224),          # Redimensiona as imagens para 224x224 pixels
    batch_size=32,                   # Tamanho do lote
    class_mode='categorical',        # Codificação one-hot (ou None se apenas previsões forem necessárias)
    shuffle=False                    # Não embaralha, útil para manter a ordem original
)

In [ ]:
#Avaliação do modelo com o conjunto de teste
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Loss no conjunto de teste: {test_loss}")
print(f"Acurácia no conjunto de teste: {test_accuracy}")

In [ ]:
#Previsões
predictions = model.predict(test_generator)
predicted_classes = predictions.argmax(axis=-1)  # Classe com maior probabilidade
print(predicted_classes)

In [ ]:
# Finalizando o TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs/